In [1]:
from htmd.ui import *
from moleculekit.config import config

config(viewer='webgl')

2024-06-11 15:54:19,768 - numexpr.utils - INFO - Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-06-11 15:54:19,769 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.
2024-06-11 15:54:19,878 - rdkit - INFO - Enabling RDKit 2022.09.1 jupyter extensions



Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. https://dx.doi.org/10.1021/acs.jctc.6b00049
HTMD Documentation at: https://software.acellera.com/htmd/

You are on the latest HTMD version (2.3.28+2.g8caf4970a).



# Building Barnase - Barstar for protein-protein interactions

![barnasebarstar](img/barnasebarstar.png)

## Download the two proteins and view them

You can look at their PDB information here and find out their PDB IDs. Then download them using those IDs.

* [Barnase](http://www.rcsb.org/pdb/explore.do?structureId=2f4y)
* [Barstar](http://www.rcsb.org/pdb/explore.do?structureId=2hxx)

You will need to create Molecule objects. Check the documentation on the [Molecule](https://www.htmd.org/docs/moleculekit.molecule.html) class.

In [2]:
barnase = Molecule('2f4y')
barstar = Molecule('2hxx')

## Filter the structures to keep only one chain of each

Pick only one chain of each structure. This will keep the crystal waters corresponding to the chosen chain.

In [3]:
barnase.filter('chain A')
barstar.filter('chain A')

2024-06-11 15:54:26,540 - moleculekit.molecule - INFO - Removed 1919 atoms. 964 atoms remaining in the molecule.
2024-06-11 15:54:26,549 - moleculekit.molecule - INFO - Removed 743 atoms. 765 atoms remaining in the molecule.


array([ 719,  720,  721,  722,  723,  724,  725,  726,  727,  728,  729,
        730,  731,  732,  733,  734,  735,  736,  737,  738,  739,  740,
        741,  742,  743,  744,  745,  746,  747,  748,  749,  750,  751,
        752,  753,  754,  755,  756,  757,  758,  759,  760,  761,  762,
        763,  764,  765,  766,  767,  768,  769,  770,  771,  772,  773,
        774,  775,  776,  777,  778,  779,  780,  781,  782,  783,  784,
        785,  786,  787,  788,  789,  790,  791,  792,  793,  794,  795,
        796,  797,  798,  799,  800,  801,  802,  803,  804,  805,  806,
        807,  808,  809,  810,  811,  812,  813,  814,  815,  816,  817,
        818,  819,  820,  821,  822,  823,  824,  825,  826,  827,  828,
        829,  830,  831,  832,  833,  834,  835,  836,  837,  838,  839,
        840,  841,  842,  843,  844,  845,  846,  847,  848,  849,  850,
        851,  852,  853,  854,  855,  856,  857,  858,  859,  860,  861,
        862,  863,  864,  865,  866,  867,  868,  8

## Visualize the filtered structures

In [4]:
from ipywidgets.widgets import Box; w = []
w.append(barnase.view())
w.append(barstar.view())
Box(children=(w[0],w[1]))

Box(children=(NGLWidget(), NGLWidget()))

## Mutate modified residues

[Barstar](http://www.rcsb.org/pdb/explore/explore.do?structureId=2hxx) has a modified residue for which we lack the parametrization (check under "Small Molecules" on PDB). Mutate the modified Tryptophan in Barstar (resname 4IN) to a normal Tryptophan (TRP):

In [5]:
barstar.mutateResidue('resname 4IN', 'TRP')

## Assignments and renaming

Assign a different chain (A, B) and segment to each protein (BRN, STR):

In [6]:
barnase.set('chain', 'A', 'protein')
barstar.set('chain', 'B', 'protein')
barnase.set('segid', 'BRN')
barstar.set('segid', 'STR')

Since the crystal waters were kept, assign them to segid W1 and W2 (assigning waters of both molecules to the same segid can cause problems as they will have same resids)

In [7]:
barnase.set('segid', 'W1', 'water')
barstar.set('segid', 'W2', 'water')

## Combine the proteins and center them

Create a new molecule which will contain both other molecules.

In [8]:
mol = Molecule()
mol.append(barnase)
mol.append(barstar)

Now center the new combined molecule on the origin.

In [9]:
mol.center()

## Solvate the combined system

Find the maximum distance of the atoms from the center:

In [10]:
from moleculekit.util import maxDistance
D = maxDistance(mol); print(D)

41.728752


Add 5 &Aring; to this distance, to add some extra space in the box, and then solvate (no need to add a salt concentration):

In [11]:
D += 5
smol = solvate(mol, minmax=[[-D, -D, -D],[D, D, D]])

2024-06-11 15:54:43,847 - htmd.builder.solvate - INFO - Using water pdb file at: /home/sdoerr/Work/htmd/htmd/share/solvate/wat.pdb
2024-06-11 15:54:44,154 - htmd.builder.solvate - INFO - Replicating 8 water segments, 2 by 2 by 2
Solvating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:02<00:00,  3.74it/s]
2024-06-11 15:54:46,896 - htmd.builder.solvate - INFO - 24833 water molecules were added to the system.


## Build the solvated system in AMBER

In [12]:
molbuilt = amber.build(smol, outdir='./build/')

2024-06-11 15:54:59,039 - htmd.builder.amber - INFO - Detecting disulfide bonds.
2024-06-11 15:55:00,515 - htmd.builder.amber - INFO - Starting the build.
2024-06-11 15:55:05,251 - htmd.builder.amber - INFO - Finished building.
2024-06-11 15:55:06,228 - moleculekit.writers - WARNING - Field "resid" of PDB overflows. Your data will be truncated to 4 characters.
2024-06-11 15:55:11,444 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (TYR 158 CA  ) (TYR 158 C  ) (PRO 159 N  ) (PRO 159 CA  ) " with indexes [2452, 2469, 2471, 2481]
2024-06-11 15:55:11,930 - htmd.builder.ionize - INFO - Adding 0 anions + 4 cations for neutralizing and 0 ions for the given salt concentration 0 M.
2024-06-11 15:55:14,245 - htmd.builder.amber - INFO - Starting the build.
2024-06-11 15:55:19,291 - htmd.builder.amber - INFO - Finished building.
2024-06-11 15:55:20,267 - moleculekit.writers - WARNING - Field "resid" of PDB overflows. Your data will be trunc

## Visualize the built system

Can take a while to load...

In [13]:
molbuilt.view(sel='protein',style='NewCartoon',
              color='Secondary Structure', hold=True)
molbuilt.view(sel='water',style='lines')

2024-06-11 15:55:29,459 - moleculekit.writers - WARNING - Field "resid" of PDB overflows. Your data will be truncated to 4 characters.


NGLWidget()